In [14]:
import ete3
from ete3 import Tree
import numpy as np
import random
import json

In [11]:
# get name of the script
nameScript = "get_tree.py"
# get json file name of the script
nameJson = "model_param.json"

print("------------------------------------------------------------------------")
print("File proprocessing for 5-taxon trees")
print("------------------------------------------------------------------------")
print("Executing " + nameScript + " following " + nameJson, flush = True)

# opening Json file 
jsonFile = open(nameJson) 
dataJson = json.load(jsonFile)

data_root = dataJson["dataRoot"]         # data folder
model_root = dataJson["modelRoot"]       # folder to save the data

label_files = dataJson["labelFile"]      # file with labels
sequence_files = dataJson["matFile"]     # file with sequences
tree_files = dataJson["treeFile"]        # file with tree structure

if "summaryFile" in dataJson:
    summary_file = dataJson["summaryFile"]
else :
    summary_file = "summary_file.txt"


print("------------------------------------------------------------------------") 
print("Loading Sequence Data in " + sequence_files, flush = True)
print("Loading Label Data in " + label_files, flush = True)
print("Loading Tree Data in " + tree_files, flush = True)

# we read the labels as list of strings
with open(data_root+label_files, 'r') as f:
    label_char = f.readlines()

# we read the sequence as a list of strings
with open(data_root+sequence_files, 'r') as f:
    seq_string = f.readlines()

with open(data_root+tree_files, 'r') as f:
    tree_newick = f.readlines()

tree_newick[1]

------------------------------------------------------------------------
File proprocessing for 5-taxon trees
------------------------------------------------------------------------
Executing get_tree.py following model_param.json
------------------------------------------------------------------------
Loading Sequence Data in sequences12062021.in
Loading Label Data in labels12062021.in
Loading Tree Data in trees12062021.in


'(1:0.699176245335339,(2:0.3402578285913327,((4:0.7774982160898161,5:0.729350459883324):0.5129091418912692,3:0.032494125943784744):0.3584708485721173):0.11733277547993536);\n'

In [20]:
t = Tree(tree_newick[0])
print(t)


         /-1
      /-|
     |   \-5
   /-|
  |  |   /-2
--|   \-|
  |      \-3
  |
   \-4


In [23]:
for node in t.traverse("postorder"):
  # Do some analysis on node
  print(node.name)

1
5

2
3


4

